# Advanced NLP HW0

Before starting the task please read thoroughly these chapters of Speech and Language Processing by Daniel Jurafsky & James H. Martin:

•	N-gram language models: https://web.stanford.edu/~jurafsky/slp3/3.pdf

•	Neural language models: https://web.stanford.edu/~jurafsky/slp3/7.pdf 

In this task you will be asked to implement the models described there.

Build a text generator based on n-gram language model and neural language model.
1.	Find a corpus (e.g. http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt ), but you are free to use anything else of your interest
2.	Preprocess it if necessary (we suggest using nltk for that)
3.	Build an n-gram model
4.	Try out different values of n, calculate perplexity on a held-out set
5.	Build a simple neural network model for text generation (start from a feed-forward net for example). We suggest using tensorflow + keras for this task

Criteria:
1.	Data is split into train / validation / test, motivation for the split method is given
2.	N-gram model is implemented
a.	Unknown words are handled
b.	Add-k Smoothing is implemented
3.	Neural network for text generation is implemented
4.	Perplexity is calculated for both models
5.	Examples of texts generated with different models are present and compared
6.	Optional: Try both character-based and word-based approaches.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import softmax

import re
import urllib.request as urllib2

from collections import defaultdict
import random

import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline, padded_everygrams
from nltk.lm import MLE, Vocabulary, KneserNeyInterpolated, WittenBellInterpolated, Laplace, Lidstone



from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split

In [2]:
data = list(urllib2.urlopen('https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt'))

In [3]:
def preproc(data):
    data = [line.strip().decode("utf-8")  for line in data]
    pat = re.compile(r'((\b\w*)|(\b\w*\s?\b\w*)):$')
    data = [i.lower() for i in data if i]
    p = []
    speech = ''
    for line in data:
        if not pat.findall(line):
            if not speech:
                speech = line
            else:
                speech += ' ' + line

        else:
            p.append(speech)
            speech = ''
    p = [string for string in p if len(string) != 0]
    
    return p

In [4]:
data = preproc(data)

In [5]:
appos = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"im" :"I am",
"isn't" : "is not",
"its": "it is",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"won't":"will not",
"didn't": "did not",
"'t'": ' it', 
"'em": "them",
"o'": "of", 
"'ll": " will",
"ne'er":"never",
"'ld": " would", "i'": "in",
"'d": "ed", 
"'en ": "ken ", 
"'bout":"about", 
"'gainst":"against", 
"'scape":"escape", 
"'mongst": "amongst", 
"'n": "en", 
"e'er":"ever", 
"itwas":"it was"
}
for i, j in appos.items():
    for k in range(len(data)):
        data[k] = data[k].replace(i, j)   

In [6]:
tokenized = list(map(nltk.word_tokenize, data))

In [7]:
tokenized[0:4]

[['before',
  'we',
  'proceed',
  'any',
  'further',
  ',',
  'hear',
  'me',
  'speak',
  '.'],
 ['speak', ',', 'speak', '.'],
 ['you',
  'are',
  'all',
  'resolved',
  'rather',
  'to',
  'die',
  'than',
  'to',
  'famish',
  '?'],
 ['resolved', '.', 'resolved', '.']]

## Models

Base class for the model.

In [8]:
X_train, X_test = train_test_split(tokenized, test_size=0.01, random_state=42)

In [9]:
class BaseLM:
    
    def __init__(self, n, gamma, vocab = None):
    
        """Language model constructor
        n -- n-gram size
        vocab -- optional fixed vocabulary for the model
        """
        self.n = n
        self.vocab = vocab
        self.corpus = []
        self.dic = defaultdict(lambda: defaultdict(lambda: 0))
        self.gamma = gamma
        self.generate_corpus()
        
    def generate_corpus(self):
        
        """
        The formula for add-k smoothing is:

        p(w_i|w_i-1) = (c(w_i-1, w-i) + k) / (c(w_i-1) + k*V),

        where V is the number of unique  words in the vocabulary

        """
      
        rare_words = pd.Series([item for speech in self.vocab for item in speech]).value_counts()[(pd.Series([item for speech in self.vocab for item in speech]).value_counts() < 2)].index.tolist()
        rare_words_dict = {k: "<UNK>" for  k in rare_words}
        self.vocab_unk = [list(map(lambda x: rare_words_dict[x] if x in rare_words_dict.keys() else x, [item for item in speech])) for speech in self.vocab]
            
        for speech in self.vocab_unk:

            ngram = nltk.ngrams([word for word in speech], self.n, pad_right=True, pad_left=True)
            self.corpus.append(list(ngram))

        N = len([item for speech in self.corpus for item in speech])
        V = len(set([item for speech in self.vocab_unk for item in speech]))
            
        for ngram in [item for sublist in self.corpus for item in sublist]:
            self.dic[(ngram[:-1])][ngram[-1]] += 1

        for key in self.dic.keys():
            total = float(sum(self.dic[key].values()))
            for value in self.dic[key]:
                self.dic[key][value] = (self.dic[(key)][value] + self.gamma) / (total + self.gamma*V)


        print("The length of the vocabulary is {}".format(V))
        print("The number of the {}-grams is {}".format(self.n, N))

             

    def prob(self, word, context=None):
        """This method returns probability of a word with given context: P(w_t | w_{t - 1}...w_{t - n + 1})

        For example:
        >>> lm.prob('hello', context=('world',))
        0.99988
        """
        V = len(set([item for speech in self.vocab_unk for item in speech]))
        if word in self.dic[tuple(context)].keys():
            ans = self.dic[tuple(context)][word]
        elif "<UNK>" in self.dic[tuple(context)].keys():
            ans = self.dic[tuple(context)]["<UNK>"]
        else:
            total = float(sum(self.dic[tuple(context)].values()))
            ans = (self.gamma) / (total + self.gamma*V)
        
        return ans
 
    def generate_text(self, text_length):
        """This method generates random text of length 

        For example
        >>> lm.generate_text(2)
        hello world

        """
        text = list(list(self.dic.keys())[random.randint(0, len(self.dic))])
        scaler = StandardScaler()
        
        while len(text)<=text_length:
  
            probs = scaler.fit_transform(np.array(list(self.dic[tuple(text[-(self.n-1):])].values())).reshape(-1, 1))
            probs = softmax(probs).reshape(-1)
            new_word = np.random.choice(list(self.dic[tuple(text[-(self.n-1):])].keys()), p=probs)
            text.append(new_word)
                
        for ind, word in enumerate(text):
            if ind % 10 == 0:
                text.insert(ind, "\n")
            
        text_to_display = ' '.join([w for w in text if w])
        for mark in re.findall(r"\s[.;:,!?\\']", text_to_display):
            text_to_display= text_to_display.replace(mark, mark[-1:])
        
        print(text_to_display)
                

    def update(self, sequence_of_tokens):
        """This method learns probabiities based on given sequence of tokents

        sequence_of_tokens -- iterable of tokens

        For example
        >>> lm.update(['hello', 'world'])
        """
        self.vocab.extend(sequence_of_tokens)
        self.generate_corpus()
        
        
    
    def perplexity(self, sequence_of_tokens):
        """This method returns perplexity for a given sequence of tokens

        sequence_of_tokens -- iterable of tokens
        """
        test_corpus = []
        for speech in sequence_of_tokens:

            ngram = nltk.ngrams([word for word in speech], self.n, pad_right=True, pad_left=True)
            test_corpus.append(list(ngram))

        entropy = -1* np.mean([np.log2(blm.prob(ngram[-1], ngram[:-1])) for ngram in [item for speech in test_corpus for item in speech]])
        perplexity = pow(2, entropy)

        return perplexity

In [35]:
blm = BaseLM(2, 0.001, X_train)

The length of the vocabulary is 14427
The number of the 2-grams is 947737


In [36]:
blm.prob('assist', 'gods')

6.931447979482915e-05

In [37]:
blm.generate_text(50)


 pacified: i have you, and, and 
, and, and, and, and, 
 and, and, and the <UNK>, and 
, and, and, and the <UNK>, 
 and, and, and i am, and 
, and i am, and


In [38]:
blm.perplexity(X_test[:10])

126.60719326192819

In [39]:
blm = BaseLM(3, 0.001, X_train)

The length of the vocabulary is 14427
The number of the 3-grams is 981814


In [40]:
blm.prob('you', ('gods', 'assist'))

0.060936263468679606

In [41]:
blm.generate_text(500)


 prospero be living and die. i 
 am not; for, i am a gentleman 
 of great worth and honour. i 
 will not be so, i will not be 
 so, my lord, i will not be 
 so, my lord, i will not be 
 excused. i will not be a 
 <UNK>, and i will not be uplifted. 
 but, i am not; but, as 
 i am a gentleman? i am 
 not; stand aside. i will 
 not be a <UNK>, and the <UNK> of 
 the world, with his own <UNK>. 
 i am not, sir, i will 
 not be a man, i am a gentleman 
 of blood and death will have a daughter called 
 katharina. i am not; i 
 will not be a <UNK>, and i will 
 not be a <UNK>, and i will not 
 be so, i am a gentleman. come 
, come, come, come, come, 
 come, come, come, come, come 
, come, come they to have redress against 
 them, and the <UNK> of the world, 
 i will not be so, my lord, 
 i am a man of <UNK>, and i 
 will not be so, sir, i will 
 not be a <UNK>, and i will not 
 be a <UNK> rudesby full of <UNK>, and 
 the <UNK> of the world, in the world 
, he is a good man's life, 
 and i wil

In [42]:
blm.perplexity(X_test[:10])

331.5135812006942

In [43]:
blm = BaseLM(4, 0.001, X_train)

The length of the vocabulary is 14427
The number of the 4-grams is 1015891


In [44]:
blm.prob('you', ('the', 'gods', 'assist'))

0.0648862384131717

In [46]:
blm.generate_text(500)


 with this thin helm? mine enemy's dog 
, and that is the way to make an 
 ass of me? i am 
 not what i am about, till candles and 
 starlight and moonshine be out. 
 i will not be long; meantI ame, 
 but hearts for the event. 
 i will not, hold me still; the 
 tI ame, and, in the name lay 
 a moiety of my estate to your ring, 
 i will not be long; that the property 
 of youth and bloom of lustihood. 
 i am not <UNK> here? 
 i will not. i 
 am glad to see you. 
 i am not a hilding, hold me your 
 loyal servant, your physician, your most dear 
 daughter -- i am for you 
. i am nothing: but 
 whatever i be, nor goodly ilion stand; 
 our firebrand brother, paris, burns us all 
. i am, and bring 
 hI am to the tower, and was fidele 
. what think you? i 
 am not mad: i would not wish a 
 drop of blood in me i 
 am not well in health, and that is 
 myself. give me some help here, ho 
? i think you are not, which, 
 when it stands well with her. ah, 
 sirrah! quoth-a, we shall not marry till

In [47]:
blm.perplexity(X_test[:10])

1832.0852398798609